In [1]:
# khai báo mạng NN bằng nn.Squential
import torch
import torch.nn as nn
from torch.nn import functional as F

In [ ]:
# với cách khai báo này, ta không cần xây dựng lớp kế thừa nn.Module
# mà chỉ cần sử dụng lớp nn.Sequential để xếp chồng các lớp với nhau, thư viện sẽ tự tính toán bước forwward và backward

net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])

In [ ]:
# khi muốn mô hình phức tạp hơn, tạo 1 class kế thừa từ nn.Module
# và cần định nghĩa hai hàm chính: __init__: khai báo các layer sẽ dùng
# và forward: cách dữ liệu đi qua các layer đó như thế nào
# ưu điểm của cách này là linh hoạt, cài đạt được các mô hình phức tạp hơn(CNN, RNN...)
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256) # Khai báo
        self.out = nn.LazyLinear(10)

    def forward(self, X):
        # Định nghĩa luồng xử lý
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X).shape    
#tuy nhiên ta vẫn có thể dùng nn.Sequential bên trong lớp kế thừa nn.Module

torch.Size([2, 10])

In [ ]:
#module Squential hoạt động như sau:
# chúng ta thêm từng mô đun bằng phương thức add_module
# các mô đun được lưu trữ theo thứ tự chúng được thêm vào và có thể được truy cập thông qua phương thức children
# trong phương thức forward, ta lặp qua từng mô đun con và áp dụng chúng tuần tự cho đầu vào X
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X
    
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

In [12]:
# lớp squential giúp việc xây dựng mô hình trở nên dễ dàng, cho phép chúng ta lắp ráp các kiến trúc mới mà không cần tự định nghĩa lớp riêng
#tuy nhiên có cá kiến trúc phức tạp hơn, cần thực hiện các phép toán tùy ý chứ không dựa vào các lớp nn đã được định nghĩa trước

# Chúng ta có thể kết hợp nhiều cách khác nhau để lắp ráp các mô-đun lại với nhau.
# Trong ví dụ sau, chúng ta sẽ lồng ghép các mô-đun theo một số cách sáng tạo.

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)


tensor(-0.1507, grad_fn=<SumBackward0>)

In [16]:
# mô đun song song

import torch
from torch import nn

class ParallelModule(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1
        self.net2 = net2

    def forward(self, x):
        # Tính toán song song
        out1 = self.net1(x)
        out2 = self.net2(x)
        
        # Nối kết quả lại với nhau.
        # dim=1 thường là chiều features (dim=0 là batch size)
        return torch.cat((out1, out2), dim=1)

# --- Kiểm thử ---
# Giả sử net1 ra 10 đặc trưng, net2 ra 5 đặc trưng
net1 = nn.Linear(20, 10)
net2 = nn.Linear(20, 5)

model = ParallelModule(net1, net2)
x = torch.randn(2, 20) # Batch size = 2, Input features = 20

output = model(x)
print(output) 
# Kết quả mong đợi: torch.Size([2, 15]) (vì 10 + 5 = 15)
model.state_dict()

tensor([[ 0.2151, -0.5308, -0.2259,  0.5902,  0.2067,  0.9651, -0.4999, -1.2603,
          0.8167, -0.4721,  0.2084, -0.6674, -0.2071,  0.3930, -0.0323],
        [-0.3996, -0.8507, -0.4302, -0.9287,  0.2898,  0.1542,  0.0946,  0.5835,
          0.1162,  0.3871,  0.0266, -0.3982,  0.5140, -0.2887,  0.5076]],
       grad_fn=<CatBackward0>)


OrderedDict([('net1.weight',
              tensor([[-1.9397e-02,  4.7253e-04,  7.7692e-02, -9.2073e-02,  1.3591e-01,
                        4.3496e-02,  1.6036e-01,  1.9428e-01,  1.9738e-01, -3.7790e-02,
                        5.6512e-03, -3.7642e-02,  1.7422e-01,  1.9512e-01,  1.2719e-01,
                       -1.0816e-01,  1.3560e-01,  5.3789e-04,  7.3464e-02,  9.4677e-02],
                      [-1.1808e-01, -2.1036e-02,  7.3819e-02,  2.1821e-01,  1.9521e-02,
                       -9.5672e-02,  5.9470e-02,  5.7355e-03,  6.4888e-02, -1.3629e-01,
                        1.8840e-01, -1.0153e-01, -5.6176e-02, -6.8100e-03,  1.4118e-01,
                        1.9593e-01,  2.4483e-02,  7.2193e-02,  1.7772e-01,  1.8065e-02],
                      [-1.7216e-02, -6.7794e-02,  1.1102e-01,  1.7209e-01,  1.6185e-01,
                        1.5640e-01, -2.9872e-02,  5.2600e-02, -1.0563e-01, -1.1272e-01,
                       -6.8660e-02, -2.1458e-01,  7.6510e-02,  1.6430e-01, -5.1042e-02,
 